In [44]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import numpy as np

In [5]:
hop_proj=sqlite3.connect('data/hop_proj.sqlite')

query = """
WITH entity1_nash AS (
    SELECT *
    FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE cbsa = 34980
        AND entity_code = 1),
entity2_nash AS (
    SELECT *
    FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE cbsa = 34980
        AND entity_code = 2)
SELECT 
    from_npi,
    to_npi,
    patient_count,
    transaction_count,
    e1.class_spec,
    e2.org_name
FROM hop AS h 
JOIN entity1_nash as e1 
ON h.from_npi = e1.npi 
JOIN entity2_nash as e2 
ON h.to_npi = e2.npi
WHERE transaction_count >= 50 
    AND average_day_wait < 50 
    AND e1.class_spec NOT LIKE '%Pathology%' 
ORDER BY transaction_count DESC
"""

test = pd.read_sql(query, hop_proj)

In [6]:
test

,from_npi,to_npi,patient_count,transaction_count,class_spec,org_name
0,1124020011,1104854124,8428,13890,Anesthesiology(Pain Medicine),ANESTHESIA SERVICES ASSOCIATES PLLC
1,1417131715,1245393057,5645,9242,Internal Medicine(Interventional Cardiology),CENTENNIAL HEART LLC
2,1417131715,1023055126,5571,9007,Internal Medicine(Interventional Cardiology),"HCA HEALTH SERVICES OF TENNESSEE, INC."
3,1306822234,1073585774,6979,8525,Dermatology,PATHOLOGY ASSOCIATES OF ST. THOMAS
4,1174517593,1104202761,1610,7054,Internal Medicine(Hematology & Oncology),VANDERBILT UNIVERSITY MEDICAL CENTER
...,...,...,...,...,...,...
46271,1073549176,1649228735,35,50,Specialist,TENNESSEE ONCOLOGY PLLC
46272,1144311580,1467687822,33,50,Psychiatry & Neurology(Psychiatry),"CENTENNIAL PSYCHIATRIC ASSOCIATES, LLC"
46273,1316204423,1275942013,40,50,Physical Medicine & Rehabilitation,LONGS PARK EMERGENCY PHYSICIANS PLLC
46274,1588089551,1457323164,47,50,Nurse Practitioner(Family),MINUTECLINIC DIAGNOSTIC OF TENNESSEE PC


In [16]:
filtered = test[test['class_spec'].str.contains("Pathology")]

In [3]:
hop_proj=sqlite3.connect('data/hop_proj.sqlite')

query = """
  WITH indiv AS (
    SELECT npi, postal_code, entity_code, org_name AS ind_org, last_name, first_name, class_spec AS indiv_spec
    FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 1
        AND cbsa = 34980
),
org_novandy AS (
    SELECT npi, postal_code, entity_code, org_name, class_spec AS org_spec
     FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 2
        AND cbsa = 34980
        and org_name NOT LIKE '%vanderbilt%'
)
SELECT h.from_npi, h.to_npi, h.transaction_count, i.indiv_spec, i.postal_code AS zip
FROM hop AS h 
JOIN indiv as i
ON h.from_npi = i.npi
JOIN org_novandy AS o_n
ON h.to_npi = o_n.npi
"""

vanderbilt = pd.read_sql(query, hop_proj)

vanderbilt

,from_npi,to_npi,transaction_count,indiv_spec,zip
0,1801075247,1245233220,193,"Nurse Anesthetist, Certified Registered",37203
1,1427098912,1245233220,133,Anesthesiology,37203
2,1336415819,1245233220,76,Anesthesiology,37203
3,1619332400,1245233220,182,"Nurse Anesthetist, Certified Registered",37203
4,1689875130,1245233220,60,Anesthesiology,37203
...,...,...,...,...,...
42115,1982829297,1891281069,134,Family Medicine,37091
42116,1376635052,1891281069,53,Family Medicine,38474
42117,1780687830,1407333842,80,Pathology(Cytopathology),37055
42118,1760832661,1407333842,70,Nurse Practitioner(Psychiatric/Mental Health),37064


In [4]:
query = """
WITH indiv AS (
    SELECT npi, postal_code, entity_code, org_name AS ind_org, last_name, first_name, class_spec AS indiv_spec
    FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 1
        AND cbsa = 34980
),
org_novandy AS (
    SELECT npi, postal_code, entity_code, org_name, class_spec AS org_spec
     FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 2
        AND cbsa = 34980
        and org_name LIKE '%vanderbilt%'
)
SELECT h.from_npi, h.to_npi, h.transaction_count, i.indiv_spec, i.postal_code AS zip
FROM hop AS h 
JOIN indiv as i
ON h.from_npi = i.npi
JOIN org_novandy AS o_n
ON h.to_npi = o_n.npi
"""

only_vanderbilt = pd.read_sql(query, hop_proj)

only_vanderbilt

,from_npi,to_npi,transaction_count,indiv_spec,zip
0,1528174752,1598738205,52,Internal Medicine(Cardiovascular Disease),37232
1,1770725905,1598738205,55,Radiology(Neuroradiology),37232
2,1780812289,1598738205,97,Radiology(Diagnostic Radiology),37232
3,1548437437,1598738205,68,Radiology(Diagnostic Radiology),37232
4,1588899215,1598738205,63,Radiology(Diagnostic Radiology),37232
...,...,...,...,...,...
5334,1417291550,1285091330,409,Nurse Practitioner(Adult Health),37232
5335,1043229495,1972027159,149,Internal Medicine(Nephrology),37232
5336,1548464027,1972027159,68,Internal Medicine(Nephrology),37232
5337,1669554556,1972027159,75,Internal Medicine(Nephrology),37232


In [24]:
hop_proj=sqlite3.connect('data/hop_proj.sqlite')

query = """
WITH indiv AS (
    SELECT npi AS ind_npi, postal_code, org_name AS ind_org, last_name, first_name, class_spec AS indiv_spec
    FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE cbsa = 34980
),
all_org AS (
    SELECT npi AS org_npi, postal_code AS org_zip, org_name, class_spec AS org_spec
     FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 2
        AND cbsa = 34980
),
org_vandy AS (
    SELECT *
    FROM all_org 
    WHERE org_name LIKE '%vanderbilt%'
),
org_novandy AS (
    SELECT *
    FROM all_org 
    WHERE org_name NOT LIKE '%vanderbilt%'
),
ind_novandy AS (
    SELECT *
    FROM hop AS h 
    JOIN indiv as i
    ON h.from_npi = i.ind_npi
    JOIN org_novandy AS ov
    ON h.to_npi = ov.org_npi  
),
ind_vandy AS (
    SELECT *
    FROM hop AS h 
    JOIN indiv as i
    ON h.from_npi = i.ind_npi
    JOIN org_vandy AS o_n
    ON h.to_npi = o_n.org_npi  
),
zip_novandy AS (
    SELECT postal_code, org_spec, SUM(transaction_count) AS no_vandy_sum
    FROM ind_novandy
    GROUP BY 1, 2
),
zip_vandy AS (
SELECT postal_code, org_spec, SUM(transaction_count) AS vandy_sum
    FROM ind_vandy
)
SELECT 
    postal_code,
    org_spec,
    no_vandy_sum,
    vandy_sum,
    (vandy_sum * 100.0 / (no_vandy_sum + vandy_sum)) AS perc_vandy_rec
FROM zip_novandy AS a_nv
LEFT JOIN zip_vandy AS z_v
USING(postal_code, org_spec);    
"""

percentages = pd.read_sql(query, hop_proj)

In [25]:
percentages

,postal_code,org_spec,no_vandy_sum,vandy_sum,perc_vandy_rec
0,37012,Family Medicine,758,NaN,NaN
1,37012,General Acute Care Hospital,110,NaN,NaN
2,37012,Internal Medicine,151,NaN,NaN
3,37012,Radiology(Diagnostic Radiology),69,NaN,NaN
4,37013,Ambulance,351,NaN,NaN
...,...,...,...,...,...
3695,38563,Internal Medicine(Nephrology),127,NaN,NaN
3696,38563,Nurse Practitioner(Family),1737,NaN,NaN
3697,38563,Pharmacy(Community/Retail Pharmacy),155,NaN,NaN
3698,38563,Physical Therapist,589,NaN,NaN


In [111]:
hop_proj=sqlite3.connect('data/hop_proj.sqlite')

query = """
WITH indiv AS (
    SELECT npi AS ind_npi, postal_code, last_name, first_name, class_spec AS from_spec
    FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 1
        AND cbsa = 34980
)
SELECT *
FROM hop AS h 
JOIN indiv as i
ON h.from_npi = i.ind_npi  
    
"""
from_eone = pd.read_sql(query, hop_proj)

In [112]:
from_eone

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,ind_npi,postal_code,last_name,first_name,from_spec
0,1376544767,1003052697,47,68,20.721,26.386,1376544767,37190,SPURLOCK,JAMES,Family Medicine
1,1376544767,1003230236,138,171,39.456,57.050,1376544767,37190,SPURLOCK,JAMES,Family Medicine
2,1376544767,1003863580,68,77,45.169,49.092,1376544767,37190,SPURLOCK,JAMES,Family Medicine
3,1376544767,1073592762,18,59,17.424,43.223,1376544767,37190,SPURLOCK,JAMES,Family Medicine
4,1376544767,1093741464,161,202,41.153,64.348,1376544767,37190,SPURLOCK,JAMES,Family Medicine
...,...,...,...,...,...,...,...,...,...,...,...
118502,1417382151,1376635052,14,65,6.738,7.787,1417382151,37036,LOVELL,ROBYN,Social Worker(Clinical)
118503,1417382151,1386803310,17,162,0.000,0.000,1417382151,37036,LOVELL,ROBYN,Social Worker(Clinical)
118504,1417382151,1417011008,18,125,7.040,7.610,1417382151,37036,LOVELL,ROBYN,Social Worker(Clinical)
118505,1417382151,1457753162,33,123,11.577,18.136,1417382151,37036,LOVELL,ROBYN,Social Worker(Clinical)


In [113]:
query = """
WITH all_org AS (
    SELECT npi AS org_npi, org_name, class_spec AS to_spec
     FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 2
        AND cbsa = 34980
),
org_vandy AS (
    SELECT *
    FROM all_org 
    WHERE org_name LIKE '%vanderbilt%'
)
SELECT *
FROM hop AS h 
JOIN org_vandy AS o_v
ON h.to_npi = o_v.org_npi  
    
"""

to_etwo = pd.read_sql(query, hop_proj)

In [114]:
to_etwo['from_to_npi'] = to_etwo['from_npi'].astype(str) + "-" + to_etwo['to_npi'].astype(str)
from_eone['from_to_npi'] = from_eone['from_npi'].astype(str) + "-" + from_eone['to_npi'].astype(str)
to_etwo

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,org_npi,org_name,to_spec,from_to_npi
0,1528174752,1598738205,51,52,31.231,43.073,1598738205,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,1528174752-1598738205
1,1770725905,1598738205,54,55,21.655,17.820,1598738205,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,1770725905-1598738205
2,1780812289,1598738205,94,97,28.763,27.295,1598738205,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,1780812289-1598738205
3,1548437437,1598738205,67,68,28.088,33.668,1598738205,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,1548437437-1598738205
4,1578509998,1598738205,96,100,7.750,33.233,1598738205,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,Rehabilitation Hospital,1578509998-1598738205
...,...,...,...,...,...,...,...,...,...,...
9533,1669554556,1972027159,30,75,23.080,54.034,1972027159,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center(End-Stage Renal Disease (ESRD) T...,1669554556-1972027159
9534,1104202761,1972027159,71,332,2.190,5.395,1972027159,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center(End-Stage Renal Disease (ESRD) T...,1104202761-1972027159
9535,1124024922,1972027159,41,85,41.929,45.792,1972027159,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center(End-Stage Renal Disease (ESRD) T...,1124024922-1972027159
9536,1992897326,1972027159,19,51,17.588,47.181,1972027159,VANDERBILT UNIVERSITY MEDICAL CENTER,Clinic/Center(End-Stage Renal Disease (ESRD) T...,1992897326-1972027159


In [115]:
len(from_eone['from_to_npi'].unique())

118507

In [116]:
test = from_eone.merge(to_etwo, how = 'inner', left_on = 'from_to_npi', right_on = 'from_to_npi' )

In [117]:
test = test[['from_npi_x', 'to_npi_x', 'to_spec', 'patient_count_x', 'transaction_count_x', 'postal_code']]

In [118]:
to_vandy = test.pivot_table(values='transaction_count_x', index = ['postal_code', 'to_spec'], aggfunc=np.sum).reset_index()

In [119]:
to_vandy

,postal_code,to_spec,transaction_count_x
0,37013,General Acute Care Hospital,1008
1,37013,Internal Medicine,1648
2,37015,General Acute Care Hospital,248
3,37015,Internal Medicine,367
4,37025,General Acute Care Hospital,169
...,...,...,...
130,38401,Internal Medicine,27649
131,38474,General Acute Care Hospital,583
132,38474,Internal Medicine,1365
133,38563,General Acute Care Hospital,72


In [120]:
query = """
WITH all_org AS (
    SELECT npi AS org_npi, org_name, class_spec AS to_spec
     FROM nppes AS n 
    JOIN cbsa AS c 
    ON n.postal_code = c.zip
    JOIN taxonomy AS t 
    ON n.primary_taxonomy = t.code
    WHERE entity_code = 2
        AND cbsa = 34980
),
org_vandy AS (
    SELECT *
    FROM all_org 
    WHERE org_name NOT LIKE '%vanderbilt%'
)
SELECT *
FROM hop AS h 
JOIN org_vandy AS o_v
ON h.to_npi = o_v.org_npi  
    
"""

to_etwo_novandy = pd.read_sql(query, hop_proj)

In [121]:
to_etwo_novandy['from_to_npi'] = to_etwo_novandy['from_npi'].astype(str) + "-" + to_etwo_novandy['to_npi'].astype(str)
test_b = from_eone.merge(to_etwo_novandy, how = 'inner', left_on = 'from_to_npi', right_on = 'from_to_npi')
test_b = test_b[['from_npi_x', 'to_npi_x', 'to_spec', 'patient_count_x', 'transaction_count_x', 'postal_code']]
to_not_vandy = test_b.pivot_table(values='transaction_count_x', index = ['postal_code', 'to_spec'], aggfunc=np.sum).reset_index()

In [122]:
to_not_vandy['zip_to_spec'] = to_not_vandy['postal_code'] + '-' + to_not_vandy['to_spec']
to_vandy['zip_to_spec'] = to_vandy['postal_code'] + '-' + to_vandy['to_spec']



In [146]:
percentages = to_not_vandy.merge(to_vandy, how = 'left', on = 'zip_to_spec').fillna(0)

In [147]:
percentages = percentages[['postal_code_x', 'to_spec_x', 'transaction_count_x_x', 'transaction_count_x_y']]
percentages = percentages.rename(columns={'postal_code_x':'prov_zip', 'to_spec_x':'ref_spec', 'transaction_count_x_x':'non_vandy_transactions', 'transaction_count_x_y':'vandy_transactions'})

In [148]:
percentages['perc_vandy_recs'] = percentages['vandy_transactions']/(percentages['vandy_transactions'] + percentages['non_vandy_transactions']) * 100

In [149]:
query = """
SELECT DISTINCT class_spec
FROM taxonomy
JOIN nppes
ON code = primary_taxonomy
WHERE org_name LIKE '%vanderbilt%'
    
"""

vandy_specs = pd.read_sql(query, hop_proj)

vandy_specs

,class_spec
0,Allergy & Immunology
1,Family Medicine
2,Internal Medicine
3,Internal Medicine(Interventional Cardiology)
4,Radiology(Diagnostic Radiology)
5,Radiology(Radiation Oncology)
6,Surgery
7,Counselor(Mental Health)
8,Psychologist
9,Social Worker(Clinical)


In [150]:
vandy_services = vandy_specs['class_spec'].tolist()

for i in vandy_services:
    percentages_cond = percentages[percentages['ref_spec'].str.contains('i')]

In [151]:
percentages_cond

,prov_zip,ref_spec,non_vandy_transactions,vandy_transactions,perc_vandy_recs
0,37013,Chiropractor,185,0.0,0.0
1,37013,Clinic/Center(Ambulatory Surgical),4310,0.0,0.0
2,37013,Clinic/Center(Mental Health (Including Communi...,204,0.0,0.0
3,37013,Clinic/Center(Physical Therapy),135,0.0,0.0
4,37013,Clinical Medical Laboratory,1092,0.0,0.0
...,...,...,...,...,...
2642,38563,Internal Medicine(Nephrology),64,0.0,0.0
2643,38563,Nurse Practitioner(Family),1737,0.0,0.0
2644,38563,Pharmacy(Community/Retail Pharmacy),80,0.0,0.0
2645,38563,Physical Therapist,589,0.0,0.0


In [152]:
percentages_cond.to_csv('data/ref_by_zip.csv')